## Import necessary modules and data

In [2]:
# popraw importy 
import numpy as np
import pandas as pd
import sklearn
import keras
import seaborn
import matplotlib.pyplot as plt
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
#!pip install -q keras_metrics
import keras_metrics
from sklearn.model_selection import cross_validate
#!pip install livelossplot
from livelossplot import PlotLossesKeras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import precision_recall_fscore_support as score
import random


Using TensorFlow backend.


## Wczytujemy poprzednio utworzone zbiory

In [3]:
X = pd.read_csv('../data/cleaned_data.csv', index_col=0)
y = pd.read_csv('../data/labels.csv', header = None)

X_test = pd.read_csv('../data/X_test.csv', index_col=0)
X_train = pd.read_csv('../data/X_train.csv', index_col=0)
X_train_std = pd.read_csv('../data/X_train_std.csv', index_col=0)
X_test_std = pd.read_csv('../data/X_test_std.csv', index_col=0)
y_train = pd.read_csv('../data/y_train.csv', index_col=0)
y_test = pd.read_csv('../data/y_test.csv', index_col=0)

Zbuduję przykładową sieć, która składa się z warstwy wejściowej, jednej warstwy ukrytej i  warstwy wyjściowej.
Kompilujemy model przy użyciu 

loss_function='binary_crossentropy';

optimizer='Adam'.


Naszą główną metryką będzie **precyzja** oznaczająca procent osób prawidłowie określonych jako zagrożone odejściem.

## Pierwsze podejście do modelu

In [4]:
model = Sequential()

model.add(Dense(11, activation='relu', input_shape=(10,)))
 
model.add(Dense(121, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[keras_metrics.precision(),'accuracy'])
                   
model.fit(X_train_std, y_train,epochs=3, batch_size=1, verbose=1, validation_data=(X_test,y_test))

model.evaluate(X_test_std,y_test)

Train on 6700 samples, validate on 3300 samples
Epoch 1/3
6700/6700 [==============================] - 22s 3ms/step - loss: 0.4148 - precision: 0.6655 - acc: 0.8257 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 2/3
6700/6700 [==============================] - 19s 3ms/step - loss: 0.3612 - precision: 0.7425 - acc: 0.8519 - val_loss: 3.3281 - val_precision: 0.0000e+00 - val_acc: 0.7930
Epoch 3/3
3300/3300 [==============================] - 0s 80us/step


[0.3608325544270602, 0.6681127981197152, 0.8433333333333334]

## Sprawdźmy skuteczność naszej predykcji raportem klasyfikacyjnym


In [4]:
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test,y_pred)
report1 = classification_report(y_test,y_pred,output_dict=True)
print(report)
print(report1['0']['precision'])


              precision    recall  f1-score   support

           0       0.85      0.97      0.91      2628
           1       0.78      0.34      0.47       672

   micro avg       0.85      0.85      0.85      3300
   macro avg       0.81      0.66      0.69      3300
weighted avg       0.84      0.85      0.82      3300

0.8522954091816367


### Sprawdźmy, jak prezentują się wyniki w zależności od liczby neuronów w warstwie ukrytej.

In [5]:
nodes = [10, 30, 50, 70, 90, 110]
wyniki = {}
for i in nodes:
    model = 0

    model = Sequential()

    model.add(Dense(11, activation='relu', input_shape=(10,)))

    model.add(Dense(i, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[keras_metrics.precision(), 'accuracy'])

    model.fit(X_train_std, y_train, epochs=5, batch_size=1, verbose=1, validation_data=(X_test, y_test))
    print("Zakończono trenowanie modelu z {0} węzłami.".format(i))
    model.evaluate(X_test_std, y_test)
    y_pred = model.predict_classes(X_test_std)
    report = classification_report(y_test, y_pred, output_dict=True)
    wyniki.update({i: [report['0']['precision'], report['1']['precision']]})

print(wyniki)


Train on 6700 samples, validate on 3300 samples
Epoch 1/5
6700/6700 [==============================] - 20s 3ms/step - loss: 0.4463 - precision: 0.5332 - acc: 0.8009 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 2/5
6700/6700 [==============================] - 21s 3ms/step - loss: 0.3698 - precision: 0.7334 - acc: 0.8499 - val_loss: 4.3105 - val_precision: 0.2488 - val_acc: 0.7309
Epoch 3/5
6700/6700 [==============================] - 21s 3ms/step - loss: 0.3577 - precision: 0.7666 - acc: 0.8549 - val_loss: 4.8452 - val_precision: 0.2445 - val_acc: 0.6970
Epoch 4/5
6700/6700 [==============================] - 19s 3ms/step - loss: 0.3520 - precision: 0.7695 - acc: 0.8560 - val_loss: 3.2965 - val_precision: 0.0000e+00 - val_acc: 0.7948
Epoch 5/5
6700/6700 [==============================] - 19s 3ms/step - loss: 0.3462 - precision: 0.7818 - acc: 0.8610 - val_loss: 6.8317 - val_precision: 0.1365 - val_acc: 0.5703
Zakończono trenowanie modelu z 10 węzłami.
3300/3300 [

## Wybieramy model z 90 węzłami - dał najlepsze wyniki

In [6]:
from keras.callbacks import EarlyStopping

model = Sequential()

model.add(Dense(10, activation='relu', input_shape=(10,)))
model.add(Dropout(0.1, ))

model.add(Dense(90, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[keras_metrics.precision(), 'accuracy'])

early_stopping_monitor = EarlyStopping(patience=2)
model.fit(X_train_std, y_train, epochs=30, batch_size=1, verbose=1, validation_data=(X_test, y_test),
          callbacks=[early_stopping_monitor])

model.evaluate(X_test_std, y_test)
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test, y_pred)
report1 = classification_report(y_test, y_pred, output_dict=True)
print(report)



Train on 6700 samples, validate on 3300 samples
Epoch 1/30
6700/6700 [==============================] - 22s 3ms/step - loss: 0.4434 - precision: 0.6061 - acc: 0.8099 - val_loss: 8.6511 - val_precision: 0.2399 - val_acc: 0.4579
Epoch 2/30
6700/6700 [==============================] - 21s 3ms/step - loss: 0.3893 - precision: 0.7113 - acc: 0.8382 - val_loss: 8.6462 - val_precision: 0.2402 - val_acc: 0.4582
Epoch 3/30
6700/6700 [==============================] - 29s 4ms/step - loss: 0.3697 - precision: 0.7417 - acc: 0.8485 - val_loss: 12.6619 - val_precision: 0.2041 - val_acc: 0.2058
Epoch 4/30
3300/3300 [==============================] - 0s 35us/step
              precision    recall  f1-score   support

           0       0.86      0.98      0.91      2628
           1       0.81      0.36      0.50       672

   micro avg       0.85      0.85      0.85      3300
   macro avg       0.84      0.67      0.71      3300
weighted avg       0.85      0.85      0.83      3300



## Zapisujemy wytrenowany model

### autor: Tomasz Sołtysiak